# Bulk Labelling as a Notebook

This notebook contains a convenient pattern to cluster and label new text data. The end-goal is to discover intents that might be used in a virtual assistant setting. This can be especially useful in an early stage and is part of the "iterate on your data"-mindset. 

## Dependencies 

You'll need to install a few things to get started. 

- [whatlies](https://rasahq.github.io/whatlies/)
- [human-learn](https://koaning.github.io/human-learn/)

You can install both tools by running this line in an empty cell; 

```python
%pip install "whatlies[sentence_tfm]" "human-learn"
```

We use `whatlies` to fetch embeddings and to handle the dimensionality reduction. We use `human-learn` for the interactive labelling interface. Feel free to check the documentation of both packages to learn more. 

## Let's go

To get started we'll first import a few tools.

In [1]:
import pathlib 
import numpy as np
from whatlies.language import CountVectorLanguage, UniversalSentenceLanguage, BytePairLanguage, SentenceTFMLanguage
from whatlies.transformers import Pca, Umap

Next we will load in some embedding frameworks. There can be very heavy, just so you know! 

In [2]:
lang_cv  = CountVectorLanguage(10)
lang_use = UniversalSentenceLanguage("large")
lang_bp  = BytePairLanguage("en", dim=300, vs=200_000)
lang_brt = SentenceTFMLanguage('distilbert-base-nli-stsb-mean-tokens')

INFO:absl:Using /var/folders/0v/pj9vtxhd6ml7mb8n94wvcmkr0000gp/T/tfhub_modules to cache modules.


Next we'll load in the texts that we'd like to embed/cluster. Feel free to provide another file here.

In [3]:
txt = pathlib.Path("nlu.md").read_text()
texts = list(set([t.replace(" - ", "") for t in txt.split("\n") if len(t) > 0 and t[0] != "#"]))
print(f"We're going to plot {len(texts)} texts.")

Keep in mind that it's better to start out with 1000 sentences or so. Much more might break the browser's memory in the next visual.

## Showing Clusters 

The cell below will take the texts and have them pass through different language backends. After this they will be mapped to a two dimensional space by using [UMAP](https://umap-learn.readthedocs.io/en/latest/). It takes a while to plot everything (mainly because the universal sentence encoder and the transformer language models are heavy).

In [6]:
%%time

def make_plot(lang):
    return (lang[texts]
             .transform(Umap(2))
             .plot_interactive(annot=False)
             .properties(width=200, height=200, title=type(lang).__name__))

make_plot(lang_cv) | make_plot(lang_bp) | make_plot(lang_use) | make_plot(lang_brt)

CPU times: user 2min 54s, sys: 1min 23s, total: 4min 18s
Wall time: 1min 25s


alt.HConcatChart(...)

What you see are four charts. You should notice that certain clusters have appeared. For your usecase you might need to check which language backend makes the most sense. 

## Note for Non-English 

The only model shown here that is English specific is the universal sentence encoder (`lang_use`). All the other ones also support other languages. For more information check the [bytepair documentation](https://nlp.h-its.org/bpemb/) and the [sentence transformer documentation](https://www.sbert.net/docs/pretrained_models.html#multi-lingual-models).

## Towards Labelling 

We'll now prepare a dataframe that we'll assign labels to. We'll do that by loading in the same text file but now into a pandas dataframe.

In [7]:
df = lang_use[texts].transform(Umap(2)).to_dataframe().reset_index()
df.columns = ['text', 'd1', 'd2']
df['label'] = ''
df.shape[0]

1087

We are now going to be labelling!

# Fancy interactive drawing! 

We'll be using Vincent's infamous [human-learn library](https://koaning.github.io/human-learn/guide/drawing-features/custom-features.html) for this. First we'll need to instantiate some charts.

Next we get to draw! Drawing can be a bit tricky though, so pay attention. 

1. You'll want to double-click to start drawing. 
2. You can then click points together to form a polygon. 
3. Next you need to double-click to stop drawing. 

This allows you to draw polygons that can be used in the code below to fetch the examples that you're interested in.

## Rerun

This is where we will start labelling. That also means that we might re-run this cell after we've added labels.

In [12]:
from hulearn.experimental.interactive import InteractiveCharts

charts = InteractiveCharts(df.loc[lambda d: d['label'] == ''], labels=['group'])

charts.add_chart(x='d1', y='d2')

Loading BokehJS ...

We can now use this selection to retreive a subset of rows. This is a quick varification to see if the points you select indeed belong to the same cluster.

In [15]:
from hulearn.preprocessing import InteractivePreprocessor
tfm = InteractivePreprocessor(json_desc=charts.data())

df.pipe(tfm.pandas_pipe).loc[lambda d: d['group'] != 0].head(10)

,text,d1,d2,label,group
27,"Hi, I need the time.",0.288001,9.669730,,1
57,What's the current time?,0.029014,9.020774,,1
82,what is the time?,0.302297,9.029532,,1
108,what time is it,0.173853,9.194384,,1
126,What is the current time?,0.046603,9.026630,,1
127,What time is it right now?,0.080451,9.046416,,1
151,Could you tell me the time?,0.122013,9.677414,,1
166,Could you tell me what time is it?,0.068536,9.423045,,1
174,What time do we have?,0.304732,9.039809,,1
183,what time do you have?,0.230429,9.116360,,1


If you're confident that you'd like to assign a label, you can do so below. 

In [16]:
label_name = 'weather'

In [19]:
idx

Int64Index([  27,   57,   82,  108,  126,  127,  151,  166,  174,  183,  219,
             235,  281,  300,  322,  323,  339,  388,  406,  414,  446,  452,
             501,  510,  532,  571,  637,  644,  651,  663,  683,  711,  724,
             740,  785,  792,  797,  848,  853,  865,  871,  880,  894,  942,
             943,  972,  979,  988,  992, 1071, 1072],
           dtype='int64')

In [21]:
idx = df.pipe(tfm.pandas_pipe).loc[lambda d: d['group'] != 0].index

df.iloc[idx, 3] = label_name

print(f"We just assigned {len(idx)} labels!")

We just assigned 51 labels!


That's it! You've just attached a label to a group of points! 

## Rerun 

You can now scroll up and start relabelling clusters that aren't assigned yet. Once you're confident that this works, you can export by running the final code below.

In [22]:
df.head()

,text,d1,d2,label
0,What languages can you communicate in?,2.727605,-0.036231,
1,what u can do?,10.641682,5.367977,
2,What exactly is my name?,14.029011,1.462056,
3,What does Rasa make?,24.505444,6.782425,
4,can you help me?,8.584467,4.582135,


In [168]:
df.to_csv("first_order_labelled.csv")

## Final Notes

There's a few things to mention. 

1. This method of labelling is great when you're working on version 0 of something. It'll get you a whole lot of data fast but it won't be high quality data. 
2. The use-case for this method might be at the start of design a virtual assistant. You've probably got data from social media that you'd like to use as a source of inspiration for intents. This is certainly a valid starting point but you should be aware that the language that folks use on a feedback form is different than the language used in a chatbox. Again, these labels are a reasonable starting point, but they should not be regarded as ground truth. 
3. Labelling is only part of the goal here. Another big part is understanding the data. This is very much a qualitative/human task. You might be able to quickly label 1000 points in 5 minutes with this technique but you'll lack an understanding if you don't take the time for it. 